In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rd-kit/rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train.csv
/kaggle/input/neurips-open-polymer-prediction-2025/test.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv


In [2]:
!pip install /kaggle/input/rd-kit/rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

Processing /kaggle/input/rd-kit/rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [3]:
import pandas as pd
train_csv = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train.csv")
train_csv['SMILES'][1]

'*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5ccc(N*)cc5)cc4)CCC(CCCCC)CC3)cc2)cc1'

In [4]:
test_csv = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/test.csv")
test_csv

,id,SMILES
0,1109053969,*Oc1ccc(C=NN=Cc2ccc(Oc3ccc(C(c4ccc(*)cc4)(C(F)...
1,1422188626,*Oc1ccc(C(C)(C)c2ccc(Oc3ccc(C(=O)c4cccc(C(=O)c...
2,2032016830,*c1cccc(OCCCCCCCCOc2cccc(N3C(=O)c4ccc(-c5cccc6...


In [5]:
Tc_extra = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv")
test2 = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv")
Tg_extra = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv")
FFV_extra = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv")

Tc_extra.rename(columns={"TC_mean": "Tc"}, inplace=True)
Tc_extra

,SMILES,Tc
0,*/C(=C(\c1ccccc1)c1ccc(*)cc1)c1ccccc1,0.3380
1,*/C(F)=C(\F)C(F)(C(*)(F)F)C(F)(F)F,0.1020
2,*/C=C(/*)C#CCCCCCCCCCCCCCCCCCCCCC(=O)O,0.4105
3,*/C=C(/*)CCCCCCCCCCCCCCCCCCCCC(=O)O,0.4030
4,*/C=C/*,0.5260
...,...,...
869,CCNC(=O)OCCCCC(*)=C=C=C(*)CCCCOC(=O)NCC,0.2600
870,Cc1ccc(cc1)S(=O)(=O)OCCCCC(*)=C=C=C(*)CCCCOS(=...,0.2100
871,FC(F)(F)C(C1=CC2=C(OC(=N2)C2=CC=C(OC3=C4C5C6=C...,0.1090
872,FC(F)(F)C(C1=CC2=C(OC(=N2)C2=CC=C(OC3=C4C5C6=C...,0.1760


In [54]:
def smiles_to_features(smiles, fp_size=1024, radius=2):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            raise ValueError("Invalid SMILES")

        # --- 18 molecular descriptors ---
        desc_list = [
            Descriptors.MolWt(mol), Descriptors.MolLogP(mol), Descriptors.TPSA(mol),
            Descriptors.NumRotatableBonds(mol), Descriptors.NumHDonors(mol),
            Descriptors.NumHAcceptors(mol), Descriptors.RingCount(mol),
            Descriptors.HeavyAtomCount(mol), Descriptors.FractionCSP3(mol),
            Descriptors.NumAliphaticRings(mol), Descriptors.NumAromaticRings(mol),
            Descriptors.LabuteASA(mol), Descriptors.MolMR(mol),
            Descriptors.NumValenceElectrons(mol), Descriptors.MaxPartialCharge(mol),
            Descriptors.MinPartialCharge(mol), Descriptors.BalabanJ(mol),
            Descriptors.BertzCT(mol)
        ]

        # Replace NaN/inf with 0
        desc_list = [0.0 if (np.isnan(x) or np.isinf(x)) else x for x in desc_list]
        desc_arr = np.array(desc_list, dtype=float)

        # --- Fingerprint (ECFP) ---
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=fp_size)
        fp_arr = np.zeros((fp_size,), dtype=int)
        DataStructs.ConvertToNumpyArray(fp, fp_arr)

        # --- Invalid flag = 0 for valid molecules ---
        invalid_flag = np.array([0.0], dtype=float)

        return np.concatenate([desc_arr, fp_arr, invalid_flag])

    except Exception:
        # --- Return consistent size: 18 + fp_size + 1 ---
        return np.concatenate([np.zeros(fp_size + 18, dtype=float), [1.0]])


In [55]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, DataStructs
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


In [56]:
non_null_count = train_csv['FFV'].notnull().sum()
print(non_null_count) 

7030


In [57]:
features = ['SMILES','FFV']
FFV_train_ = train_csv[train_csv['FFV'].notnull()][features].copy()

features = ['SMILES','Tg']
Tg_train_ = train_csv[train_csv['Tg'].notnull()][features].copy()

features = ['SMILES','Tc']
Tc_train_ = train_csv[train_csv['Tc'].notnull()][features].copy()

features = ['SMILES','Density']
Density_train = train_csv[train_csv['Density'].notnull()][features].copy()

features = ['SMILES','Rg']
Rg_train = train_csv[train_csv['Rg'].notnull()][features].copy()


In [58]:
Tc_train_

,SMILES,Tc
0,*CC(*)c1ccccc1C(=O)OCCCCCC,0.205667
10,*c1ccc(-c2ccc3c(c2)C(CCCCCCC#N)(CCCCCCC#N)c2cc...,0.487000
11,*CC(*)c1ccc(C(=O)O)c(C(=O)O)c1,0.171000
31,*CCCCCNC(=O)CCCCC(=O)N*,0.327000
34,*CCCCCCCCCCCCCCCCCCNC(=O)NCCCCCCNC(=O)N*,0.383000
...,...,...
7899,*CCN(*)C(=O)CCCCCCCCCCCCCCC,0.336000
7905,*CC(*)c1ccccc1F,0.174667
7943,*CC(*)C1CC=CCC1,0.180750
7944,*CC(*)Cl,0.139000


In [59]:
FFV_train = pd.concat([FFV_train_, FFV_extra], ignore_index = True)
Tg_train = pd.concat([Tg_train_, Tg_extra], ignore_index = True)
Tc_train = pd.concat([Tc_train_, Tc_extra], ignore_index = True)
print(FFV_train.shape)
print(Tg_train.shape)
print(Tc_train.shape)
print(Density_train.shape)
print(Rg_train.shape)

(7892, 2)
(557, 2)
(1611, 2)
(613, 2)
(614, 2)


In [60]:
# Example: build X and y
X = []
y = []

for smi, target in zip(FFV_train["SMILES"], FFV_train["FFV"]):
    feats = smiles_to_features(smi)
    if feats is not None:
        X.append(feats)
        y.append(target)

X = np.array(X)
y = np.array(y)

print("Feature matrix shape:", X.shape)


Feature matrix shape: (7892, 1043)


In [13]:
from sklearn.model_selection import train_test_split
Y = FFV_train['FFV'].values
X_train_FFV, X_test_FFV, Y_train_FFV, Y_test_FFV = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)

In [14]:
#Trying XGBOOST
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
model_FFV = xgb.XGBRegressor(
    n_estimators=3500,
    learning_rate=0.04,
    max_depth=9,
    subsample=0.7,
    colsample_bytree=0.5634168569532054,
    reg_lambda=2,    
    reg_alpha=0.01,   
    random_state=42,
    tree_method="hist"
)

In [15]:
model_FFV.fit(
    X_train_FFV, Y_train_FFV,
    eval_set=[(X_test_FFV, Y_test_FFV)],
    eval_metric="mae",
    early_stopping_rounds=50,
    verbose=50
)
Y_pred_FFV = model_FFV.predict(X_test_FFV)
print("Validation MAE:", mean_absolute_error(Y_test_FFV, Y_pred_FFV))

[0]	validation_0-mae:0.02013


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[50]	validation_0-mae:0.00922
[100]	validation_0-mae:0.00746
[150]	validation_0-mae:0.00692
[200]	validation_0-mae:0.00660
[250]	validation_0-mae:0.00641
[300]	validation_0-mae:0.00626
[350]	validation_0-mae:0.00615
[400]	validation_0-mae:0.00606
[450]	validation_0-mae:0.00599
[500]	validation_0-mae:0.00592
[550]	validation_0-mae:0.00587
[600]	validation_0-mae:0.00583
[650]	validation_0-mae:0.00579
[700]	validation_0-mae:0.00576
[750]	validation_0-mae:0.00573
[800]	validation_0-mae:0.00570
[850]	validation_0-mae:0.00568
[900]	validation_0-mae:0.00567
[950]	validation_0-mae:0.00566
[1000]	validation_0-mae:0.00565
[1050]	validation_0-mae:0.00564
[1100]	validation_0-mae:0.00563
[1150]	validation_0-mae:0.00563
[1200]	validation_0-mae:0.00562
[1250]	validation_0-mae:0.00562
[1300]	validation_0-mae:0.00561
[1350]	validation_0-mae:0.00561
[1400]	validation_0-mae:0.00561
[1450]	validation_0-mae:0.00560
[1500]	validation_0-mae:0.00560
[1550]	validation_0-mae:0.00560
[1600]	validation_0-mae:0.00

In [16]:
X_sub = []

for smi in test_csv["SMILES"]:
    feats = smiles_to_features(smi)
    if feats is not None:
        X_sub.append(feats)

X_sub = np.array(X_sub)

print("sub matrix shape:", X_sub.shape)

sub matrix shape: (3, 1043)


In [17]:
Y_FFV_sub = model_FFV.predict(X_sub)
Y_FFV_sub

array([0.37094736, 0.37526122, 0.3503764 ], dtype=float32)

In [52]:
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, rdMolDescriptors
from rdkit import DataStructs
import numpy as np

def smiles_to_Tg_features(smiles, fp_size=1024, radius=2):
    """
    Extract features for Glass Transition Temperature (Tg) prediction:
    - Physicochemical and topological descriptors
    - Morgan fingerprint (ECFP, radius=2, nBits=fp_size)
    """
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            raise ValueError("Invalid SMILES")

        # --- Tg-related molecular descriptors (15 total) ---
        desc_list = [
            Descriptors.MolWt(mol),                     
            Descriptors.MolLogP(mol),                   
            Descriptors.TPSA(mol),                      
            Descriptors.NumRotatableBonds(mol),         
            Descriptors.NumHDonors(mol),                
            Descriptors.NumHAcceptors(mol),             
            Descriptors.RingCount(mol),                 
            Descriptors.HeavyAtomCount(mol),            
            Descriptors.FractionCSP3(mol),              
            Descriptors.NumAliphaticRings(mol),         
            Descriptors.NumAromaticRings(mol),          
            Descriptors.LabuteASA(mol),                 
            rdMolDescriptors.CalcChi0n(mol),            
            rdMolDescriptors.CalcKappa2(mol),           
            rdMolDescriptors.CalcNumLipinskiHBA(mol),   
        ]
        desc_arr = np.array(desc_list, dtype=float)

        # --- Morgan fingerprint ---
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=fp_size)
        fp_arr = np.zeros((fp_size,), dtype=int)
        DataStructs.ConvertToNumpyArray(fp, fp_arr)

        # --- Combine ---
        return np.concatenate([desc_arr, fp_arr])

    except Exception:
        # return zero vector of same length (15 + fp_size)
        return np.zeros(15 + fp_size, dtype=float)



In [53]:
X_Tg = []
Y_Tg = []

for smi, target in zip(Tg_train["SMILES"], Tg_train["Tg"]):
    feats = smiles_to_Tg_features(smi)
    if feats is not None:
        X_Tg.append(feats)
        Y_Tg.append(target)

X_Tg = np.array(X_Tg)
Y_Tg = np.array(Y_Tg)

print("Feature matrix shape:", X_Tg.shape)

Feature matrix shape: (557, 1039)


In [20]:
#Trying XGBOOST
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
model_Tg = xgb.XGBRegressor(
    n_estimators=1500,
    learning_rate=0.05,
    max_depth=7,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1,    
    reg_alpha=0.12,   
    random_state=42,
    tree_method="hist"
)
Y_Tg = Tg_train['Tg'].values
X_train_Tg, X_test_Tg, Y_train_Tg, Y_test_Tg = train_test_split(X_Tg, Y_Tg, test_size=0.2, random_state=42,shuffle=True)


In [21]:
model_Tg.fit(
    X_train_Tg, Y_train_Tg,
    eval_set=[(X_test_Tg, Y_test_Tg)],
    eval_metric="mae",
    early_stopping_rounds=50,
    verbose=50
)

[0]	validation_0-mae:82.62844


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[50]	validation_0-mae:54.96365
[100]	validation_0-mae:55.01184
[108]	validation_0-mae:54.93660


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1500, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [22]:
Y_pred_Tg = model_Tg.predict(X_test_Tg)
print("Validation MAE:", mean_absolute_error(Y_test_Tg, Y_pred_Tg))

Validation MAE: 54.6660267042521


In [50]:
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem
from rdkit import DataStructs
import numpy as np

def smiles_to_Tc_features(smiles, fp_radius=2, fp_bits=512):
    """
    Extract features for Critical Temperature (Tc) prediction:
    - Physicochemical descriptors
    - Connectivity indices
    - Morgan fingerprints (bit vector of length fp_bits)
    """
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            raise ValueError("Invalid SMILES")

        # --- Basic physico-chemical descriptors ---
        mol_wt = Descriptors.MolWt(mol)
        logp = Descriptors.MolLogP(mol)
        tpsa = Descriptors.TPSA(mol)
        rot_bonds = Descriptors.NumRotatableBonds(mol)
        h_donors = Descriptors.NumHDonors(mol)
        h_acceptors = Descriptors.NumHAcceptors(mol)
        aromatic_proportion = sum(atom.GetIsAromatic() for atom in mol.GetAtoms()) / (mol.GetNumAtoms() + 1e-6)

        # --- Connectivity / topology descriptors ---
        balaban_j = Descriptors.BalabanJ(mol)
        chi0 = Descriptors.Chi0(mol)
        chi1 = Descriptors.Chi1(mol)
        chi2 = Descriptors.Chi2n(mol)

        descriptor_features = np.array([
            mol_wt, logp, tpsa, rot_bonds,
            h_donors, h_acceptors, aromatic_proportion,
            balaban_j, chi0, chi1, chi2
        ], dtype=float)

        # --- Morgan Fingerprint ---
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=fp_radius, nBits=fp_bits)
        fp_arr = np.zeros((fp_bits,), dtype=int)
        DataStructs.ConvertToNumpyArray(fp, fp_arr)

        # --- Combine ---
        return np.concatenate([descriptor_features, fp_arr])

    except Exception:
        # fallback: zero vector of correct size
        return np.zeros(11 + fp_bits, dtype=float)


In [51]:
X_Tc = []
Y_Tc = []

for smi, target in zip(Tc_train["SMILES"], Tc_train["Tc"]):
    feats = smiles_to_Tc_features(smi)
    if feats is not None:
        X_Tc.append(feats)
        Y_Tc.append(target)

X_Tc = np.array(X_Tc)
Y_Tc = np.array(Y_Tc)

print("Feature matrix shape:", X_Tc.shape)

Feature matrix shape: (1611, 523)


In [25]:
#Trying XGBOOST

model_Tc = xgb.XGBRegressor(
    n_estimators=3500,
    learning_rate=0.06,
    max_depth=12,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.3,    
    reg_alpha=0.04,   
    random_state=42,
    tree_method="hist"
)

Y_Tc = Tc_train['Tc'].values
X_train_Tc, X_test_Tc, Y_train_Tc, Y_test_Tc = train_test_split(X_Tc, Y_Tc, test_size=0.2, random_state=42,shuffle=True)


In [26]:
model_Tc.fit(
    X_train_Tc, Y_train_Tc,
    eval_set=[(X_test_Tc, Y_test_Tc)],
    eval_metric="mae",
    early_stopping_rounds=50,
    verbose=50
)
Y_pred_Tc = model_Tc.predict(X_test_Tc)
print("Validation MAE:", mean_absolute_error(Y_test_Tc, Y_pred_Tc))

[0]	validation_0-mae:0.07090


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[50]	validation_0-mae:0.01934
[100]	validation_0-mae:0.01550
[150]	validation_0-mae:0.01415
[200]	validation_0-mae:0.01348
[250]	validation_0-mae:0.01299
[300]	validation_0-mae:0.01264
[350]	validation_0-mae:0.01239
[400]	validation_0-mae:0.01220
[450]	validation_0-mae:0.01208
[500]	validation_0-mae:0.01198
[550]	validation_0-mae:0.01186
[600]	validation_0-mae:0.01176
[650]	validation_0-mae:0.01168
[700]	validation_0-mae:0.01162
[750]	validation_0-mae:0.01155
[800]	validation_0-mae:0.01150
[850]	validation_0-mae:0.01145
[900]	validation_0-mae:0.01143
[950]	validation_0-mae:0.01137
[1000]	validation_0-mae:0.01132
[1050]	validation_0-mae:0.01130
[1100]	validation_0-mae:0.01125
[1150]	validation_0-mae:0.01124
[1200]	validation_0-mae:0.01119
[1250]	validation_0-mae:0.01117
[1300]	validation_0-mae:0.01114
[1350]	validation_0-mae:0.01111
[1400]	validation_0-mae:0.01109
[1450]	validation_0-mae:0.01105
[1500]	validation_0-mae:0.01104
[1550]	validation_0-mae:0.01101
[1600]	validation_0-mae:0.01

In [46]:
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem
from rdkit import DataStructs
import numpy as np

def smiles_to_Density_features(smiles, fp_radius=2, fp_bits=512):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            raise ValueError("Invalid SMILES")

        # Basic molecular descriptors
        mol_wt = Descriptors.MolWt(mol)                   
        logp = Descriptors.MolLogP(mol)                   
        tpsa = Descriptors.TPSA(mol)                      
        mol_vol = rdMolDescriptors.CalcExactMolWt(mol) / (mol_wt + 1e-6)  
        heavy_atoms = Descriptors.HeavyAtomCount(mol)
        rot_bonds = Descriptors.NumRotatableBonds(mol)
        h_donors = Descriptors.NumHDonors(mol)
        h_acceptors = Descriptors.NumHAcceptors(mol)
        aromatic_ratio = sum(a.GetIsAromatic() for a in mol.GetAtoms()) / (mol.GetNumAtoms() + 1e-6)

        # Topological descriptors
        balaban_j = Descriptors.BalabanJ(mol)
        chi0 = Descriptors.Chi0(mol)
        chi1 = Descriptors.Chi1(mol)
        chi2 = Descriptors.Chi2n(mol)

        # Clean NaN / Inf
        features = np.array([
            mol_wt, logp, tpsa, mol_vol,
            heavy_atoms, rot_bonds, h_donors, h_acceptors,
            aromatic_ratio, balaban_j, chi0, chi1, chi2
        ], dtype=float)
        features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)

        # Fingerprint (Morgan)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=fp_radius, nBits=fp_bits)
        fp_arr = np.zeros((fp_bits,), dtype=int)
        DataStructs.ConvertToNumpyArray(fp, fp_arr)

        # Final concatenated vector
        return np.concatenate([features, fp_arr.astype(float)])

    except Exception:
        # Always return fixed-size vector (13 desc + fp_bits)
        return np.zeros(13 + fp_bits, dtype=float)


In [47]:
X_Density = []
Y_Density = []

for smi, target in zip(Density_train["SMILES"], Density_train["Density"]):
    feats = smiles_to_Density_features(smi)
    if feats is not None:
        X_Density.append(feats)
        Y_Density.append(target)

X_Density = np.array(X_Density)
Y_Density = np.array(Y_Density)

print("Feature matrix shape:", X_Density.shape)

Feature matrix shape: (613, 525)


In [29]:
# XGBOOST
model_Density = xgb.XGBRegressor(
    n_estimators=1800,
    learning_rate=0.04,
    max_depth=9,
    subsample=0.5,
    colsample_bytree=0.8,
    reg_lambda=1,    
    reg_alpha=0.1,   
    random_state=42,
    tree_method="hist"
)

Y_Density = Density_train['Density'].values
X_train_D, X_test_D, Y_train_D, Y_test_D = train_test_split(X_Density, Y_Density, test_size=0.2, random_state=42,shuffle=True)

In [30]:
# XGBOOST
model_Density = xgb.XGBRegressor(
    n_estimators=2000,
    learning_rate=0.05,
    max_depth=9,
    subsample=0.5,
    colsample_bytree=0.8,
    reg_lambda=1,    
    reg_alpha=0.1,   
    random_state=42,
    tree_method="hist"
)

Y_Density = Density_train['Density'].values
X_train_D, X_test_D, Y_train_D, Y_test_D = train_test_split(X_Density, Y_Density, test_size=0.2, random_state=42,shuffle=True)

In [31]:
model_Density.fit(
    X_train_D, Y_train_D,
    eval_set=[(X_test_D, Y_test_D)],
    eval_metric="mae",
    early_stopping_rounds=50,
    verbose=50
)
Y_pred_D = model_Density.predict(X_test_D)
print("Validation MAE:", mean_absolute_error(Y_test_D, Y_pred_D))

[0]	validation_0-mae:0.10292


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[50]	validation_0-mae:0.04045
[100]	validation_0-mae:0.03472
[150]	validation_0-mae:0.03283
[200]	validation_0-mae:0.03198
[250]	validation_0-mae:0.03188
[300]	validation_0-mae:0.03171
[350]	validation_0-mae:0.03180
[372]	validation_0-mae:0.03175
Validation MAE: 0.031677825463851304


In [48]:
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem
import numpy as np

def smiles_to_Rg_features(smiles, fp_radius=2, fp_bits=1024):
    """
    Extracts features for Radius of Gyration (Rg) prediction:
    - 2D descriptors (size, flexibility, polarity)
    - Morgan fingerprints (bit vector of length fp_bits)
    """
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            raise ValueError("Invalid SMILES")

        # --- Descriptors (Rg proxies) ---
        mw = Descriptors.MolWt(mol)
        heavy_atoms = mol.GetNumHeavyAtoms()
        rot_bonds = Descriptors.NumRotatableBonds(mol)
        tpsa = Descriptors.TPSA(mol)
        logp = Descriptors.MolLogP(mol)
        kappa1 = rdMolDescriptors.CalcKappa1(mol)
        kappa2 = rdMolDescriptors.CalcKappa2(mol)
        kappa3 = rdMolDescriptors.CalcKappa3(mol)
        chi0 = Descriptors.Chi0(mol)
        chi1 = Descriptors.Chi1(mol)

        # crude proxy for Rg (size × flexibility)
        rg_proxy = (mw / (heavy_atoms + 1)) * (1 + rot_bonds * 0.1)

        descriptor_features = np.array([
            mw, heavy_atoms, rot_bonds, tpsa, logp,
            kappa1, kappa2, kappa3, chi0, chi1, rg_proxy
        ], dtype=float)

        # --- Morgan Fingerprints ---
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=fp_radius, nBits=fp_bits)
        fp_array = np.zeros((fp_bits,), dtype=int)
        # safer than ToBitString → directly numeric
        from rdkit import DataStructs
        DataStructs.ConvertToNumpyArray(fp, fp_array)

        # --- Combine ---
        return np.concatenate([descriptor_features, fp_array])

    except Exception:
        # fallback: zero vector of the correct size
        return np.zeros(11 + fp_bits, dtype=float)


In [49]:
X_Rg = []
Y_Rg = []

for smi, target in zip(Rg_train["SMILES"], Rg_train["Rg"]):
    feats = smiles_to_Rg_features(smi, fp_bits=1024)
    if feats is not None:
        X_Rg.append(feats)
        Y_Rg.append(target)

X_Rg = np.array(X_Rg)
Y_Rg = np.array(Y_Rg)

print("Feature matrix shape:", X_Rg.shape)

Feature matrix shape: (614, 1035)


In [34]:
# XGBOOST
model_Rg = xgb.XGBRegressor(
    n_estimators=1800,
    learning_rate=0.08,
    max_depth=9,
    subsample=0.7,
    colsample_bytree=0.7,
    reg_lambda=2,
    gamma=1,
    reg_alpha=0,   
    random_state=42,
    tree_method="hist"
)
Y_Rg = Rg_train['Rg'].values
X_train_Rg, X_test_Rg, Y_train_Rg, Y_test_Rg = train_test_split(X_Rg, Y_Rg, test_size=0.2, random_state=42,shuffle=True)

In [35]:
model_Rg.fit(
    X_train_Rg, Y_train_Rg,
    eval_set=[(X_test_Rg, Y_test_Rg)],
    eval_metric="mae",
    early_stopping_rounds=50,
    verbose=50
)
Y_pred_Rg = model_Rg.predict(X_test_Rg)
print("Validation MAE:", mean_absolute_error(Y_test_Rg, Y_pred_Rg))

[0]	validation_0-mae:3.69136


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[50]	validation_0-mae:1.51663
[100]	validation_0-mae:1.52346
[111]	validation_0-mae:1.52408
Validation MAE: 1.4933989156777425


In [36]:
#1.4933989156777425
X_sub_Tg = []

for smi in test_csv["SMILES"]:
    feats = smiles_to_Tg_features(smi)
    if feats is not None:
        X_sub_Tg.append(feats)

X_sub_Tg = np.array(X_sub_Tg)

print("sub matrix shape:", X_sub_Tg.shape)

sub matrix shape: (3, 1039)


In [37]:
Y_Tg_sub = model_Tg.predict(X_sub_Tg)
Y_Tg_sub

array([145.7898 , 178.53674,  89.0169 ], dtype=float32)

In [38]:
X_sub_Tc = []

for smi in test_csv["SMILES"]:
    feats = smiles_to_Tc_features(smi)
    if feats is not None:
        X_sub_Tc.append(feats)

X_sub_Tc = np.array(X_sub_Tc)

print("sub matrix shape:", X_sub_Tc.shape)

sub matrix shape: (3, 523)


In [39]:
Y_Tc_sub = model_Tc.predict(X_sub_Tc)
Y_Tc_sub

array([0.21225072, 0.22863652, 0.25805622], dtype=float32)

In [40]:
X_sub_D = []

for smi in test_csv["SMILES"]:
    feats = smiles_to_Density_features(smi)
    if feats is not None:
        X_sub_D.append(feats)

X_sub_D = np.array(X_sub_D)

print("sub matrix shape:", X_sub_D.shape)

sub matrix shape: (3, 525)


In [41]:
Y_Density_sub = model_Density.predict(X_sub_D)
Y_Density_sub

array([1.1430455, 1.086203 , 1.0825245], dtype=float32)

In [42]:
X_sub_Rg = []

for smi in test_csv["SMILES"]:
    feats = smiles_to_Rg_features(smi)
    if feats is not None:
        X_sub_Rg.append(feats)

X_sub_Rg = np.array(X_sub_Rg)

print("sub matrix shape:", X_sub_Rg.shape)

sub matrix shape: (3, 1035)


In [43]:
Y_Rg_sub = model_Rg.predict(X_sub_Rg)
Y_Rg_sub

array([22.582312, 20.679268, 17.771944], dtype=float32)

In [44]:
submission = pd.DataFrame({
    "id": test_csv["id"],
    "Tg" : Y_Tg_sub.ravel(),
    "FFV" : Y_FFV_sub.ravel(),
    "Tc" : Y_Tc_sub.ravel(),
    "Density": Y_Density_sub.ravel(),
    "Rg" : Y_Rg_sub.ravel()
})
submission


,id,Tg,FFV,Tc,Density,Rg
0,1109053969,145.789795,0.370947,0.212251,1.143046,22.582312
1,1422188626,178.536743,0.375261,0.228637,1.086203,20.679268
2,2032016830,89.016899,0.350376,0.258056,1.082525,17.771944


In [45]:
submission.to_csv("submission.csv", index=False)
print("Done")

Done
